The purpose of this file is to perform experiments and keep notes. I prefer to do it this way to keep the other files cleaner and minimize necessary cleanup work later.

In [1]:
from collections import defaultdict
from pprint import pprint

In [2]:
import pandas as pd
from pymongo import MongoClient

client = MongoClient()
db = client.osm
bergen = db.bergen

# df = pd.DataFrame(list(db.bergen.find()))

In [3]:
#Creating function to list documents with a given feature

def docs_with_feature(feature,max_docs=5,value=None):
    
    if value:
        query_cursor = bergen.find( { feature: value } )
    else:
        query_cursor = bergen.find( { feature: {'$exists': True} } )
    cursor_count = query_cursor.count()
    
    print(cursor_count," documents have feature.")
    
    if cursor_count < max_docs+1:
        for doc in query_cursor:
            pprint(doc)
            print('----')
            
    else:
        count = 0
        print("Printing first {0} matching documents.".format(max_docs) )
        
        for doc in query_cursor:
            count += 1
            if count < max_docs:
                pprint(doc)
                print('----')
    

In [4]:
docs_with_feature('amenity')

3423  documents have feature.
Printing first 5 matching documents.
{'_id': ObjectId('58ee99f28a51df7ed61f5595'),
 'address': {'city': 'Frekhaug',
             'housenumber': '3',
             'postcode': '5918',
             'street': 'Holtermandsvegen'},
 'amenity': 'school',
 'created': {'changeset': '25431710',
             'timestamp': '2014-09-14T13:03:49Z',
             'uid': '88164',
             'user': 'Med',
             'version': '4'},
 'id': '659034',
 'name': 'Nordhordaland folkehøgskule',
 'operator': 'Nordhordland Indremisjon',
 'pos': [60.514765, 5.2414095],
 'type': 'node',
 'website': 'http://www.skolen.info/',
 'wikipedia': 'no:Nordhordland Folkehøgskule'}
----
{'_id': ObjectId('58ee99f28a51df7ed61f5bae'),
 'access': 'permissive',
 'amenity': 'parking_entrance',
 'created': {'changeset': '37097944',
             'timestamp': '2016-02-09T10:20:29Z',
             'uid': '1965308',
             'user': 'FredrikLindseth',
             'version': '11'},
 'foot': 'permis

In [5]:
feature_list = list(db.bergen.find())

In [6]:
feature_list[1000:1001]

[{'_id': ObjectId('58ee99f28a51df7ed61f55fb'),
  'created': {'changeset': '34180559',
   'timestamp': '2015-09-22T11:02:06Z',
   'uid': '3252623',
   'user': 'Alex_K_B',
   'version': '8'},
  'highway': 'traffic_signals',
  'id': '674462',
  'pos': [60.3924968, 5.3184896],
  'type': 'node'}]

In [7]:
df['created'][1:10]

NameError: name 'df' is not defined

In [ ]:
df['address'][1:10]

In [ ]:
tmp_find_one= bergen.find_one()

for col in tmp_find_one: 
    print(col)

In [ ]:
from IPython.display import display
from IPython.display import HTML

#Getting count of documents with address field

address_query = { 'address' : {'$exists' : True } }
address_documents = db.bergen.find(address_query)
address_count = address_documents.count()

address_count

In [ ]:
address_documents = db.bergen.find(address_query)#, { 
#         'address.street': 1,'address.housenumber': 1,'address.postcode': 1,'address.city': 1
#     }
#                                   )

In [ ]:
tmp_list = list(address_documents)

In [ ]:
len(tmp_list)

In [ ]:
tmp_list[1]

In [ ]:
from pprint import pprint

In [ ]:


to_df = list()
# columns = ['street','housenumber','postcode','city']
count = 0 

for doc in tmp_list:
    record = { 'id': doc['id'], 'type': doc['type'] }
    
    for key,val in doc['address'].items():
        record[key] = val
        
    for key,val in doc['created'].items():
        record['created_' + key] = val
    
    if 'pos' in doc.keys():
        record['lat'] = doc['pos'][0]
        record['lon'] = doc['pos'][1]
    
    to_df.append(record)

In [ ]:
tmp_list[1]

In [ ]:
from collections import defaultdict
# tmp_set = set()
tmp_def_dict = defaultdict(int)
for doc in tmp_list:
    
    for k,v in doc.items():
        
#         if k in ('address','created'):
        try:
            for key,val in v.items():
#             tmp_set.add(k+'_'+key)
                tmp_def_dict[k+':'+key] += 1
#         else:
        except AttributeError:
            tmp_def_dict[k] += 1
#             tmp_set.add(k)
    

In [ ]:
tmp_def_dict

In [ ]:
tmp_def_dict.keys()

In [ ]:
for doc in bergen.find( {'name': {'$exists': True}}):
    pprint(doc)

In [ ]:
tmp_s = pd.Series(tmp_def_dict)

tmp_s.sort_values(ascending=False).to_frame('count')

In [ ]:
pd.DataFrame(tmp_def_dict,index=range(len(tmp_def_dict.keys())) )

In [ ]:
tmp_set

In [ ]:
df_address = pd.DataFrame(to_df, columns = [
        'id','street','housenumber','city','postcode','created_uid','created_user',
        'created_version','created_timestamp','created_uid',
    'lat','lon','type'] )

In [ ]:
df_address[1:10]

In [ ]:
df_address.count()

In [ ]:
values = []
for cc in db.bergen.find():
  values.append(cc['created'])

print(pd.DataFrame(values)[1:10])

In [ ]:
import pandas as pd

postcodes_per_municipality = pd.read_csv('data/Postnummerregister_ansi.tsv', encoding='utf-8',delimiter='\t',header=0, names=[
        'postal_code','postal_place','muni_number','muni_name','category'],
            dtype = {'postal_code': str, 'municipality_number': str})

postcodes_per_municipality.head(5)

In [ ]:
postcodes_per_municipality['postcodes_per_municipality'] = pd.to_numeric(postcodes_per_municipality['postal_code'])

In [ ]:
import timeit
tmp_sl = postcodes_per_municipality['postal_code']


start = timeit.timeit()
for i in tmp_sl:
    if i == 5151:
        print('series for and if equal')

end = timeit.timeit()
print(end - start)

start = timeit.timeit()

if 5140 in tmp_sl.values:
        print('series if in')
    
end = timeit.timeit()
print(end - start)

tmp_sl = list(tmp_sl)

start = timeit.timeit()
for i in tmp_sl:
    if i == 5151:
        print('list for and if equal')

end = timeit.timeit()
print(end - start)

start = timeit.timeit()

if 5140 in tmp_sl:
        print('list if in')
    
end = timeit.timeit()
print(end - start)

# for func in f:
#     f = ['value = [val[5] for col,val in dictionary.items()]', 'value = s.loc[5]', 'value = df.iloc[5]']

#     print(min(timeit.Timer(func, setup).repeat(3, 100000)))

In [ ]:
%alias_magic t timeit

import timeit
tmp_sl = postcodes_per_municipality['postal_code']


for i in tmp_sl:
    if i == 5151:
        print('series for and if equal')

%t -n1 if str(5146) in tmp_sl.values: print('series if in')

tmp_sl = list(tmp_sl)

for i in tmp_sl: 
    if i == 5151: 
        print('list for and if equal')

%t -n1 if str(5146) in tmp_sl: print('list if in')

# for func in f:
#     f = ['value = [val[5] for col,val in dictionary.items()]', 'value = s.loc[5]', 'value = df.iloc[5]']

#     print(min(timeit.Timer(func, setup).repeat(3, 100000)))

In [ ]:
tmp_sl = postcodes_per_municipality['postal_code']


tmp_sl.values

In [ ]:
#Removed cell February 9, 2017

# aggregated = bergen.aggregate([  
#         {'$match' :  { 'address': {'$exists' : True }, 'type': 'way' } },
#         { "$group" : { 
#                 "_id" : "$id","count" : { "$sum" : 1} } }
#     ])

# unique_street_count = 0

# for doc in aggregated:
# #     household_count += doc['count']
#     unique_street_count += 1

# print(unique_street_count)

#Removed cell, February 9, 2017

#EXPERIMENTAL, WILL BE REMOVED
streetnames_sorted_dict = dict(sorted(addresses_on_street.items(), key=lambda x: x[1], reverse=True))

for rec in streetnames_sorted_dict:
    if streetnames_sorted_dict[rec] == 184:
        print (rec,streetnames_sorted_dict[rec],type(rec))

#Removed cell February 9, 2017

#EXPERIMENTAL, WILL BE REMOVED
tmp_agg = db.bergen.aggregate([
         { "$group" : { "_id" : "$address.street",
                       "mset" : {
                         "$addToSet" : "$address.house_number"
                 } } },
         { "$unwind" : "$mset"},
         { "$group" : { "_id" : "$_id",
                       "count" : { "$sum" : 1} } },
         { "$sort" : { "count" : -1 } },
         { "$limit" : 10 } ] )

for doc in tmp_agg:
    print(doc)

#Removed cell February 20, 2017

#Converting list for dataframe usage
to_df = []

for dic in duplicate_addresses:
    dictionary = dict()
    
#     for key,val in dic.items():
#         dictionary[key] = val
        
#         if key == '_id':
    for key,val in dic['_id'].items():
        dictionary[key] = val
        
    dictionary['count'] = dic['count']
                
        
    to_df.append(dictionary)
    
to_df

#Removed from cleaning part February 20, 2017

            if count < 10:
                pass
            elif count == 10:
                pprint(data)
            else:
                break
        
        addr_count = 0
        for node in data:
            if (addr_count < 20) and 'address' in node.keys():-=0
                addr_count += 1
                pprint(node['address'])  

In [ ]:
from collections import defaultdict

user_count_query = bergen.aggregate( [
   {
     '$group': {
        '_id' : { 'uid': '$created.uid', 'username': '$created.user' }
           }
        },
   {
     '$group': {
        '_id': 'null',
        'count': { '$sum': 1 }
     }
   }
] )

for doc in user_count_query:
    user_count = doc['count']

average_contributions = bergen.aggregate( [
   {
          '$group': 
            {
                '_id' : 
                { 'uid': '$created.uid', 'username': '$created.user' },
                'count': { '$sum': 1 } 
            } 
    },
    { 
            '$group': 
            {
                '_id': 'null',
                'avg': { '$avg': '$count' } 
            }
    }
] )

for doc in average_contributions:
    user_average = round(doc['avg'],2)
    
grouped_users = list(bergen.aggregate([  
        { 
            "$group" : 
            { 
                "_id" : { "uid": "$created.uid", "username": "$created.user" },
                "count" : { "$sum" : 1} 
            } 
        },
        { "$sort" : { "count" : 1 } }
        ]))

user_no = 0
halfway = round(user_count / 2)
mode_dict = defaultdict(int)

for doc in grouped_users:
        user_no += 1
        val = doc['count']
        if user_no == halfway:
            user_median = val
        
        mode_dict[val] += 1

user_mode = max(mode_dict.items(), key=lambda a: a[1])
mode_percentage = round((user_mode[1] / user_count) * 100,2)
            
print("Total user count:",user_count)
print("Average contributions per user:",user_average)
print("Median contributions per user:",user_median)
print("Mode of contribution count: {0} contributors ({1}%) submitted {2} edit.".format(
    user_mode[1],mode_percentage,user_mode[0] ) )



In [ ]:
import numpy as np
from matplotlib import pyplot as plt
% matplotlib inline


mu, sigma = 100, 15
x = mu + sigma*np.random.randn(10000)

bah = np.array(list(doc['count'] for doc in grouped_users))

print(len(x))

plt.hist(bah,bins=100)


In [ ]:
s_mode_dict = pd.Series(mode_dict)

In [ ]:
df_mode_dict = pd.DataFrame(s_mode_dict,columns=['count'])

In [ ]:
df_mode_dict['contributions'] = df_mode_dict.index

In [ ]:
df_mode_dict = df_mode_dict[['contributions','count']]

In [ ]:
[2,3,4,list(range(1,10,2))]

In [ ]:
list(range(1,10,2)) + list(range(1,4))

from collections import OrderedDict

bracket_names = OrderedDict()
for item in bins:
    if item != bins[-1]:
        if item < 10:
            bracket_names[str(item)] = item  
        else:
            next = bins[bins.index(item) + 1]
            bracket_names['{0} to {1}'.format(item,next)] = item
            
bracket_names

In [ ]:
bracket_names.keys()

In [ ]:
# for key,val in (bracket_names.keys(),bracket_names.values()):
#     print(key,val)

len(bracket_names.values())

print(bins)
print(list(bracket_names.values()))

tmp_dict = dict()

for item in bins:
    bin_index = bins.index(item)
        
    try:
        tmp_dict[bracket_names[bin_index]] = item
    except IndexError:
        print(item)
    
tmp_dict

In [ ]:
df_mode_dict.dtypes

In [ ]:
str(11000)[:-3] + 'K'

In [ ]:
#Creating age brackets and removing records with missing ageb

bins = list(range(0,10,1) ) + list(range(10,100,10) ) + \
list(range(100,1000,100) ) + list(range(1000,10000,5000) ) + \
list(range(10000,50000,20000) ) + list(range(50000,151000,50000) )

bracket_names = list()
for item in bins:
    if item != bins[-1]:
        if item < 10:
            bracket_names.append(str(item+1)) 
        else:
            start = item + 1
            next = bins[bins.index(item) + 1]
            if start > 1000:
                start = str(start)[:-3] + 'K'
                next = str(next)[:-3] + 'K'
            bracket_names.append('{0} to {1}'.format(start,next))
        
categories = pd.Series(pd.cut(df_mode_dict['contributions'], bins ,
                              labels=bracket_names, include_lowest=True ) )

df_mode_dict['bracket'] = categories
#pd.cut does not work perfectly for what I want, so I have to adjust some brackets
# df_mode_dict['bracket'][df_mode_dict['contributions'] < 11] = df_mode_dict['contributions']

display(df_mode_dict.head(30))
display(df_mode_dict.tail(4))

# print("Number of passengers used for age analysis:", len(age_data))

In [ ]:
for_plot = df_mode_dict.groupby('bracket')['count'].agg('sum')

In [ ]:
for_plot

In [ ]:
import seaborn as sns

fig = plt.figure(figsize=(8,5))
ax = plt.subplot(1,1,1)

#all passengers plot
rects = for_plot.plot(kind='bar',ax=ax,color='g')
ax.set_title('Contributions per user',fontsize='large',fontweight='bold')
ax.set_xlabel('Number of contributions',fontsize='small')
ax.set_ylabel('User count',fontsize='small')
ax.set_xticklabels(for_plot.index.values,rotation='70',fontsize='x-small')

plt.show()

In [ ]:
s_mode_dict.plot.hist(bins=10)

In [ ]:
bah_list = list(bah)

bah_list.sort()

bah_list[-1]

In [ ]:
s_grouped_users = pd.Series(list(doc['count'] for doc in grouped_users))

s_grouped_users.plot(kind='hist')

data = s_grouped_users

plt.hist(data, bins=range(min(data), max(data) + 5, 10))

In [ ]:
s_grouped_users.hist(bins=[0, 10, 20, 30, 40, 50, 100])

In [ ]:
df_mode_dict.iloc[50:]

In [ ]:
from scipy import stats

stats.describe(bah)

#### Misspelled streets

In [ ]:
#Getting counts for streetnames and addresses

aggregated = bergen.aggregate([  
        {'$match' : {'address': {'$exists' : True } } },
        { "$group" : { 
                "_id" : "$address.street","count" : { "$sum" : 1} } }
    ])

household_count = 0
unique_street_count = 0
addresses_on_street = {}

for doc in aggregated:
    household_count += doc['count']
    unique_street_count += 1
    
    addresses_on_street[doc['_id']] = doc['count']

print("total addresses in Bergen:", household_count)
print("number of streetnames:", unique_street_count)

In [ ]:
#Taking a look at the streets with the most addresses

from operator import itemgetter

streetnames_sorted_dict = dict(sorted(addresses_on_street.items(), key=lambda x: x[1], reverse=True)[:10])
streetnames_sorted_list = sorted(addresses_on_street.items(), key=lambda x: x[1], reverse=True)


display(HTML("<b>Streets with most addresses on them:</b>"))

for street,count in streetnames_sorted_list[0:10]:
    print(street,count)

In [ ]:
#Checking for potential duplicate data due to misspelled street names

import difflib
from fuzzywuzzy import fuzz

def fuzzy_streets(ratio,house_count):
    
    fuzzy_matches = list()
    compare_count = 0
    
    for k1 in streetnames_sorted_list:

        if k1[0] is None:
            print("Addresses without street name:",k1[1])

        #Only comparing street names with less addresses than house_count
        elif k1[1] <= house_count:
            
            compare_count += 1

            for k2 in streetnames_sorted_list:

                if k2[0] is None:
                    pass

                elif k2[0] == k1[0]:
                    pass

                else:                    
                    
                    fuzz_ratio = fuzz.ratio(k1[0],k2[0])
                    
                    if fuzz_ratio >= ratio:
                        fuzzy_matches.append({k1: k2,"fuzz ratio": fuzz_ratio})

    print("Number of street names compared: {0} of {1}".format(compare_count,len(streetnames_sorted_list)))
    
    return fuzzy_matches

In [ ]:
potential_misspellings = fuzzy_streets(92,10)

In [ ]:
#Printing out the potential misspellings

df_potential_misspellings = pd.DataFrame(columns = [
        'high_spelling','high_count','low_spelling','low_count','fuzz_ratio'])

#Adding index to make it easier to sort out the items I need to investigate further
count = 0

for spellings in potential_misspellings:
    count += 1
    df_potential_misspellings.loc[count] = None
    for key, val in spellings.items():
        if type(key) == tuple:
            if key[1] > val[1]:
                df_potential_misspellings.loc[count]['high_spelling'] = key[0]
                df_potential_misspellings.loc[count]['low_spelling'] = val[0]
                df_potential_misspellings.loc[count]['high_count'] = key[1]
                df_potential_misspellings.loc[count]['low_count'] = val[1]

            else:
                df_potential_misspellings.loc[count]['high_spelling'] = val[0]
                df_potential_misspellings.loc[count]['low_spelling'] = key[0]
                df_potential_misspellings.loc[count]['high_count'] = val[1]
                df_potential_misspellings.loc[count]['low_count'] = key[1]
        else:
            df_potential_misspellings.loc[count]['fuzz_ratio'] = val

df_potential_misspellings.drop_duplicates().sort_values('high_spelling',ascending=True)

Contributors

In [ ]:
#Getting counts for streetnames and addresses

user_count = bergen.aggregate([  
        { "$group" : { 
                "_id" : { "uid": "$created.uid", "mset": {"username": "$created.user" } } } },
        { "$unwind" : "$mset"},
        { "$group" : { "_id" : "$uid",
                      "count" : { "$sum" : 1} } }        
    ])

# household_count = 0
# unique_street_count = 0
# addresses_on_street = {}

# for doc in aggregated:
#     household_count += doc['count']
#     unique_street_count += 1
    
#     addresses_on_street[doc['_id']] = doc['count']

# print("total addresses in Bergen:", household_count)
# print("number of streetnames:", unique_street_count)

for doc in user_count:
    print(doc)

### Plan going forward:

- Export list of duplicate addresses in two formats: full data in json/osm, addresses in 2 or 3 columns (3 with postcode)
- Wrangling file: add postcode cleaning

In [ ]:
type({'a':1})

In [ ]:
for key,val in {'type': 'node', '_id': '58ee99f28a51df7ed61f5213',
  'created': {
      'version': '3', 'uid': '114230', 'user': 'danerikk', 'changeset': '6007582', 
      'timestamp': '2010-10-10T22:30:50Z'}, 'id': '358067', 'pos': [60.5314263, 5.2552169]}.items():
    print(key,val)

In [22]:

features = defaultdict(int)
count = 0

for doc in list(bergen.find()):    

    for k,v in doc.items():
        
        if k == 'node_refs':
            count += 1
        try:
            for key,val in v.items():
#             tmp_set.add(k+'_'+key)
                features[k+':'+key] += 1
#         else:
        except AttributeError:
            if type(v) == list:
                for item in v:
                    if type(item) == dict:
                        for key,val in item.items():
                            features[k+':'+key] += 1
                    else:
                        features[k] += 1
                
            elif type(v) == str:
                features[k] += 1
                if doc['id'] == '616726':
                    print(k,v)
            elif type(v) == dict:
                print('whah')
                print(key)
                print(val)
                break
            elif k != '_id':
                print("SOMETHING FISHY")
                print(k)
                print(v)
                print(type(v))
                print('----')
                break
                
count

name Havnebrekkå
id 616726
place locality
type node


52393

In [ ]:
cow_dict = {'a':1,'b':2,'c':3}

1 in cow_dict

In [ ]:
count = 0

for doc in bergen.find({"subject":{'$exists': 1}, "subject.en":{'$exists': 0 } } ):
    count += 1
    
print(count)

In [ ]:
for feature in features.keys():
    if ':' not in feature:
        print(feature,features[feature])

In [23]:
features

defaultdict(int,
            {'FIXME': 30,
             'IATAKODE': 2,
             'ICAOKODE': 2,
             'LUFTHAVNTYPE': 2,
             'TODO': 1,
             'abandoned': 4,
             'abandoned:building': 9,
             'abutters': 91,
             'access': 577,
             'access:conditional': 1,
             'access:lanes': 4,
             'access:lanes:backward': 1,
             'address:city': 84386,
             'address:country': 145,
             'address:floor': 319,
             'address:full': 2,
             'address:housename': 17,
             'address:housenumber': 84377,
             'address:postcode': 84390,
             'address:street': 84441,
             'admin_level': 50,
             'advertising': 6,
             'aerialway': 21,
             'aeroway': 58,
             'alcohol': 1,
             'alt_name': 104,
             'amenity': 3423,
             'amenity:no': 1,
             'amperage': 3,
             'aquaculture': 1,
             '

In [24]:
for key in sorted(features,key=features.get,reverse=True):
    print(key,features[key])

pos 1257558
created:timestamp 681172
created:user 681172
created:uid 681172
id 681172
type 681172
created:changeset 681172
created:version 681172
node_refs 636272
address:street 84441
address:postcode 84390
address:city 84386
address:housenumber 84377
highway 27214
source 18457
name 18154
source:date 13311
building 11463
created_by 8539
nvdb:date 6182
nvdb:id 6077
natural 5106
maxspeed 4993
amenity 3423
foot 2301
ref 2065
leaf_type 1987
oneway 1925
surface 1282
barrier 1211
shop 1164
waterway 1006
bicycle 994
layer 969
material 892
website 870
lanes 818
leisure 812
service 778
power 757
operator 719
wheelchair 706
man_made 640
landuse 628
railway 621
crossing 610
access 577
phone 569
backrest 553
public_transport 543
lit 526
sac_scale 511
opening_hours 506
tourism 502
bridge 461
level 458
wetland 417
tunnel 403
note 383
entrance 381
colour 329
voltage 319
address:floor 319
email 318
bus 285
place 280
segregated 272
office 260
int_ref 258
species 245
fixme 235
building:levels 227
ele 22

In [ ]:
bergen.find_one({'amenity': {'$exists': True}})

In [120]:
aggregated = bergen.aggregate([  
        {'$match' :  { 'amenity': {'$exists' : True }} },
        { "$group" : { 
                "_id" : "$amenity","count" : { "$sum" : 1} } },
    { "$sort" : { "_id" : 1 } }
    ])

for amenity in aggregated:
    print(amenity)

{'_id': 'arts_centre', 'count': 4}
{'_id': 'atm', 'count': 34}
{'_id': 'bank', 'count': 18}
{'_id': 'bar', 'count': 17}
{'_id': 'bbq', 'count': 7}
{'_id': 'bench', 'count': 676}
{'_id': 'bicycle_parking', 'count': 136}
{'_id': 'bicycle_rental', 'count': 1}
{'_id': 'boat_rental', 'count': 1}
{'_id': 'boat_storage', 'count': 3}
{'_id': 'bureau_de_change', 'count': 2}
{'_id': 'bus_station', 'count': 8}
{'_id': 'cabin', 'count': 2}
{'_id': 'cafe', 'count': 87}
{'_id': 'car_rental', 'count': 3}
{'_id': 'car_sharing', 'count': 25}
{'_id': 'car_wash', 'count': 4}
{'_id': 'charging_station', 'count': 82}
{'_id': 'childcare', 'count': 2}
{'_id': 'cinema', 'count': 3}
{'_id': 'clinic', 'count': 21}
{'_id': 'clock', 'count': 5}
{'_id': 'college', 'count': 6}
{'_id': 'community_centre', 'count': 2}
{'_id': 'compressed_air', 'count': 1}
{'_id': 'concert_hall', 'count': 1}
{'_id': 'courthouse', 'count': 1}
{'_id': 'dance', 'count': 1}
{'_id': 'dentist', 'count': 27}
{'_id': 'doctors', 'count': 5}
{'

In [117]:
aggregated2 = bergen.aggregate([  
        {'$match' :  { 'leisure': {'$exists' : True }} },
        { "$group" : { 
                "_id" : "$leisure","count" : { "$sum" : 1} } },
    { "$sort" : { "count" : -1 } }
    ])

for doc in aggregated2:
    print(doc)

{'_id': 'pitch', 'count': 278}
{'_id': 'playground', 'count': 252}
{'_id': 'picnic_table', 'count': 97}
{'_id': 'sports_centre', 'count': 37}
{'_id': 'park', 'count': 36}
{'_id': 'fitness_centre', 'count': 23}
{'_id': 'marina', 'count': 11}
{'_id': 'swimming_pool', 'count': 10}
{'_id': 'tanning_salon', 'count': 9}
{'_id': 'slipway', 'count': 9}
{'_id': 'stadium', 'count': 7}
{'_id': 'track', 'count': 7}
{'_id': 'firepit', 'count': 5}
{'_id': 'swimming_area', 'count': 4}
{'_id': 'bingo', 'count': 3}
{'_id': 'common', 'count': 3}
{'_id': 'recreation_ground', 'count': 2}
{'_id': 'table', 'count': 2}
{'_id': 'fishing', 'count': 2}
{'_id': 'bowling_alley', 'count': 2}
{'_id': 'horse_riding', 'count': 2}
{'_id': 'sports_centre;swimming_pool', 'count': 1}
{'_id': 'ice_rink;track', 'count': 1}
{'_id': 'golf_course', 'count': 1}
{'_id': 'bingo_hall', 'count': 1}
{'_id': 'nature_reserve', 'count': 1}
{'_id': 'taning_salong', 'count': 1}
{'_id': 'dumpster_diving', 'count': 1}
{'_id': 'range', 'co

In [ ]:
for amenity in aggregated:
    for leisure in aggregated2:
        if leisure['_id'] == amenity['_id']:
            print(amenity)
            print(leisure)
            print('----')

In [ ]:
aggregated = bergen.aggregate([  
        {'$match' :  { 'node_refs': {'$exists' : False }} },
        { "$group" : { 
                "_id" : "$amenity","count" : { "$sum" : 1} } },
    { "$sort" : { "count" : -1 } }
    ])

for doc in aggregated:
    print(doc)

In [ ]:
docs_with_feature('name',20)

In [ ]:
for doc in bergen.find( {'name': {'$regex': 'ingo'}}):
    pprint(doc)
    print('----')

In [ ]:
docs_with_feature('gambling')

In [115]:
for doc in bergen.find( { 'amenity': {'$exists': True}, 'leisure': {'$exists': True} } ):
    pprint(doc)
    print('----')

{'_id': ObjectId('58ee9a098a51df7ed62664e4'),
 'amenity': 'dance',
 'created': {'changeset': '42063695',
             'timestamp': '2016-09-10T17:44:03Z',
             'uid': '1965308',
             'user': 'FredrikLindseth',
             'version': '2'},
 'dance': {'teaching': 'yes'},
 'id': '3843535770',
 'leisure': 'dancing',
 'level': '0',
 'name': 'International Dance Academy',
 'pos': [60.3582138, 5.1268978],
 'type': 'node'}
----
{'_id': ObjectId('58ee9a128a51df7ed62910cd'),
 'amenity': 'parking',
 'created': {'changeset': '30413308',
             'timestamp': '2015-04-22T19:34:48Z',
             'uid': '2836853',
             'user': 'frokor_import',
             'version': '2'},
 'id': '112072612',
 'leisure': 'pitch',
 'node_refs': ['1274887773',
               '1274888142',
               '1274888251',
               '1274887707',
               '1274887773'],
 'source': ['Kartverket N50', {'date': '2003-05-19'}],
 'type': 'way'}
----
{'_id': ObjectId('58ee9a158a51df7ed629af

In [ ]:
from operator import itemgetter

sorted(features.items(), key=itemgetter(1), reverse=True)

In [ ]:
# from collections import defaultdict

# features = defaultdict(int)

# for doc in list(bergen.find( { 'node_refs': {'$exists' : False } } )):    

#     for k,v in doc.items():
        
        
#         if type(v) == dict:
#             for key,val in v.items():
# #             tmp_set.add(k+'_'+key)
#                 features[k+':'+key] += 1
# #         else:
#         elif type(v) == list:
#             for item in v:
#                 if type(item) == dict:
#                     for key,val in item.items():
#                         features[k+':'+key] += 1
#                 else:
#                     features[k] += 1

#         elif type(v) == str:
#             features[k] += 1
#             if doc['id'] == '616726':
#                 print(k,v)

In [ ]:
bergen.find( { 'node_refs': {'$exists' : True } } ).count()

In [ ]:
bergen.find( { 'address.city': {'$exists' : True } } ).count()

In [ ]:
tmp_city = bergen.find( { 'address.city': {'$exists' : True } } )

count = 0

for doc in tmp_city:
    for k,v in doc.items():
        if k == 'address':
            for addr_type,value in v.items():
                if addr_type == 'city':
                    count += 1
                    
                    
count

In [ ]:
for key in sorted(features,key=features.get,reverse=True):
    print(key,features[key])

In [ ]:
for key in sorted(features,key=features.get,reverse=True):
    print(key,features[key])

In [ ]:
# node_ref_count_query = bergen.aggregate( [
#    {
#      '$group': {
#         '_id' : { 'node_refs': '$node_refs'}
#            }
#         },
#     { "$sort" : { "count" : -1 } }
# ] )
       
# for doc in node_ref_count_query:
#        print(doc)
        
# # aggregated = bergen.aggregate([  
# #         {'$match' :  { 'node_refs': {'$exists' : False }} },
# #         { "$group" : { 
# #                 "_id" : "$amenity","count" : { "$sum" : 1} } },
# #     { "$sort" : { "count" : -1 } }
# #     ])

# # for doc in aggregated:
# #     print(doc)

In [11]:
node_refs_cursor = bergen.find( { 'node_refs': {'$exists' : True } } )

node_ref_counts = defaultdict(int)

for doc in node_refs_cursor:
    for node_ref in doc['node_refs']:
        node_ref_counts[node_ref] += 1

In [12]:
count = 0

for key in sorted(node_ref_counts,key=node_ref_counts.get,reverse=True):
    count += 1
    
    if count > 100:
        break
    else:
        print(key,node_ref_counts[key])

29784574 8
659176 7
331537177 7
2958740469 7
3463353550 7
653922 7
388603237 6
3459139122 6
3462350855 6
3463351284 6
1380827404 6
3792014898 6
43522929 6
2028740048 6
3473524850 6
3463351083 6
406569423 6
3464903679 6
21397989 6
3462350892 6
3466223356 6
3466223742 6
3468102863 6
3469622707 6
3469622699 6
197829725 6
2007965551 6
1589792064 6
15992470 6
941663311 6
843232472 6
2958740440 6
3465315221 6
113924307 6
29679011 6
3463353552 6
31092129 6
3463352812 6
21632993 6
29876839 6
3694806525 6
29562543 6
3464903405 6
26082309 6
43523002 6
3465315270 6
3464903021 6
2405180799 6
3437743939 6
14917507 6
2485014527 6
3462996275 6
3464903664 6
3473490762 6
3680075086 6
29943011 6
83075348 5
3473536848 5
3461476470 5
3468151029 5
29922468 5
26082306 5
3464903428 5
3469571846 5
265629308 5
3462014229 5
3469591093 5
29598840 5
3462357486 5
3466225246 5
3463351062 5
125494576 5
3462995841 5
48881406 5
3463353304 5
188991191 5
320778501 5
3695731689 5
289974402 5
17354882 5
746497503 5
346531

In [13]:
bergen.find_one( {'id': '29784574'})

{'_id': ObjectId('58ee99f28a51df7ed61f81e3'),
 'created': {'changeset': '34017753',
  'timestamp': '2015-09-14T10:51:19Z',
  'uid': '715936',
  'user': 'Gazer75',
  'version': '6'},
 'id': '29784574',
 'pos': [60.3864274, 5.3213204],
 'type': 'node'}

In [21]:
node_refs_cursor = bergen.find( { 'node_refs': {'$exists' : True } } )

for doc in node_refs_cursor:
    for node_ref in doc['node_refs']:
        if node_ref == '29784574':
            pprint(doc)
            print('------')

{'_id': ObjectId('58ee9a118a51df7ed628ee29'),
 'created': {'changeset': '14097785',
             'timestamp': '2012-11-30T11:58:27Z',
             'uid': '715936',
             'user': 'Gazer75',
             'version': '2'},
 'highway': 'unclassified',
 'id': '4685644',
 'node_refs': ['29784573', '29784574'],
 'type': 'way'}
------
{'_id': ObjectId('58ee9a128a51df7ed6291a6a'),
 'created': {'changeset': '11749235',
             'timestamp': '2012-05-30T14:35:21Z',
             'uid': '420980',
             'user': 'NorNorth',
             'version': '1'},
 'highway': 'service',
 'id': '165510865',
 'name': 'Menneskerettighetenes plass',
 'node_refs': ['1770495753', '1769659574', '29784577', '29784574', '29784576'],
 'type': 'way'}
------
{'_id': ObjectId('58ee9a128a51df7ed6291e13'),
 'created': {'changeset': '39349421',
             'timestamp': '2016-05-16T11:00:45Z',
             'uid': '1965308',
             'user': 'FredrikLindseth',
             'version': '7'},
 'id': '194290985

In [26]:
import re

In [43]:
# if re.search(':'+'dog'),'cat:dog'):
#     print('mhm')
    
for feat in features:
    re.search('(:?dog)','cat:dog')

<_sre.SRE_Match object; span=(3, 7), match=':dog'>

In [47]:
match = re.search(r":\s(.*)",'cat:dog' )
if match:
    result = match.group(1)
else:
    result = ""

result

''

In [53]:
re.search(":dog",'cat:dog')

<_sre.SRE_Match object; span=(3, 7), match=':dog'>

In [ ]:
mystring =  "hi my name is ryan, and i am new to python and would like to learn more"
keyword = 'name'
befor_keyowrd, keyword, after_keyword = mystring.partition(keyword)
# before_keyword = 'hi my '
# keyword = name
# after_keyword = ' is ryan, and i am new to python and would like to learn more'

In [60]:
'cat:dog'.split(':')[-1] == 'dog'.split(':')[-1]

True

In [92]:
matches = dict()
exceptions = set([
    'type','structure','backward','colour','height','period', 'character', 
    'range','group','en','sector_start','sector_end','no','lanes','name',
    'forward','left','right'
])
feat_count = defaultdict(int)
feat_set = set()

for feat in features:
    split_feat = feat.split(':')[-1]
    if split_feat not in exceptions:
        feat_set.add(split_feat)

for feat in feat_set:
    match_list = list()
    for match in features:            
        if feat == match.split(':')[-1]:
            feat_count[feat] += 1
            match_list.append(match)
    if len(match_list) > 1:
        matches[feat] = match_list
            
matches

{'amenity': ['source:amenity', 'amenity'],
 'amperage': ['socket:type2:amperage',
  'socket:type1:amperage',
  'socket:chademo:amperage',
  'amperage'],
 'bicycle': ['oneway:bicycle', 'bicycle', 'monitoring:bicycle'],
 'building': ['abandoned:building', 'building'],
 'car': ['fee:car', 'car'],
 'category': ['seamark:light:category',
  'seamark:bridge:category',
  'seamark:radar_transponder:category'],
 'country': ['address:country', 'is_in:country'],
 'date': ['source:date', 'population:date', 'nvdb:date'],
 'description': ['wheelchair:description', 'description'],
 'email': ['email', 'contact:email'],
 'fax': ['contact:fax', 'fax'],
 'fee': ['fee', 'internet_access:fee'],
 'hgv': ['fee:hgv', 'hgv'],
 'highway': ['highway', 'source:highway'],
 'historic': ['railway:historic', 'historic'],
 'id': ['nvdb:id', 'id', 'no-bergen-playgrounds:id'],
 'information': ['information', 'seamark:information'],
 'level': ['isced:level', 'level'],
 'levels': ['roof:levels', 'building:levels'],
 'mater

In [83]:
for key,val in matches.items():
    if val[-7:] == 'website':
        print(key,val)

heritage:website contact:website
website contact:website
contact:website website


In [90]:
for key in sorted(feat_count,key=feat_count.get,reverse=True):
    print(key,feat_count[key])

amperage 3
right 3
date 3
category 3
left 3
part 2
url 2
country 2
website 2
levels 2
source 2
bicycle 2
position 2
id 2
ref 2
psv 2
material 2
width 2
type2 1
inscription_date 1
changeset 1
twitter 1
metal 1
wheelchair 1
flat_glass 1
plastic_bottles 1
light 1
polyester 1
fee 1
correction 1
glass 1
debit_cards 1
nb 1
vegetarian 1
grade 1
engine_oil 1
housename 1
description 1
shop 1
tesla_supercharger 1
es 1
cartons 1
nsrid 1
floor 1
car 1
cash 1
shape 1
alternative_medicine 1
he 1
notes 1
magazines 1
disposal 1
combo2 1
waterway 1
building 1
email 1
operator 1
paper 1
plasterboard 1
light_bulbs 1
note 1
small_appliances 1
da 1
green_waste 1
amenity 1
street 1
plastic_bags 1
music 1
type1 1
christmas_trees 1
chipboard 1
information 1
disabled 1
clearance_height_open 1
mount 1
paper_packaging 1
zh 1
tv_monitor 1
user 1
lv 1
newspaper 1
artwork 1
nl 1
clearance_width 1
housenumber 1
sandpit 1
ssid 1
teaching 1
none 1
electrical_appliances 1
cardboard 1
chademo 1
instagram 1
timestamp 1
w

In [97]:

way_features = defaultdict(int)

for doc in list(bergen.find({'type': 'way'})):    

    for k,v in doc.items():

        try:
            for key,val in v.items():
#             tmp_set.add(k+'_'+key)
                way_features[k+'.'+key] += 1
#         else:
        except AttributeError:
            if type(v) == list:
                for item in v:
                    if type(item) == dict:
                        for key,val in item.items():
                            way_features[k+'.'+key] += 1
                    else:
                        way_features[k] += 1
                
            elif type(v) == str:
                way_features[k] += 1
                if doc['id'] == '616726':
                    print(k,v)


In [98]:
node_features = defaultdict(int)

for doc in list(bergen.find({'type': 'node'})):    

    for k,v in doc.items():

        try:
            for key,val in v.items():
#             tmp_set.add(k+'_'+key)
                node_features[k+'.'+key] += 1
#         else:
        except AttributeError:
            if type(v) == list:
                for item in v:
                    if type(item) == dict:
                        for key,val in item.items():
                            node_features[k+'.'+key] += 1
                    else:
                        node_features[k] += 1
                
            elif type(v) == str:
                node_features[k] += 1
                if doc['id'] == '616726':
                    print(k,v)

name Havnebrekkå
id 616726
place locality
type node


In [96]:
len(node_features)

487

In [101]:
node_features

defaultdict(int,
            {'FIXME': 4,
             'IATAKODE': 2,
             'ICAOKODE': 2,
             'LUFTHAVNTYPE': 2,
             'TODO': 1,
             'abandoned.building': 6,
             'access': 242,
             'address.city': 83784,
             'address.country': 92,
             'address.floor': 317,
             'address.full': 2,
             'address.housename': 5,
             'address.housenumber': 83778,
             'address.postcode': 83790,
             'address.street': 83821,
             'advertising': 6,
             'aerialway': 2,
             'aeroway': 23,
             'alt_name': 60,
             'amenity': 2407,
             'amenity.no': 1,
             'amperage': 3,
             'artist_name': 128,
             'artwork_subject': 44,
             'artwork_type': 178,
             'atm': 11,
             'authentication.none': 1,
             'automatic': 1,
             'backrest': 525,
             'barrier': 573,
             'bench': 59

In [122]:
for feat in node_features:
    if 'inter' in feat:
        print(feat)
        print('---')

internet_access.type
---
internet_access.ssid
---
internet_access
---
internet_access.fee
---
recycling.printer_cartridges
---


In [107]:
docs_with_feature('surface')

1282  documents have feature.
Printing first 5 matching documents.
{'_id': ObjectId('58ee9a068a51df7ed62587e5'),
 'created': {'changeset': '32867494',
             'timestamp': '2015-07-25T09:49:05Z',
             'uid': '992708',
             'user': 'Christian Madsen',
             'version': '2'},
 'highway': 'crossing',
 'id': '3643958450',
 'pos': [60.3090102, 5.3528302],
 'surface': 'asphalt',
 'traffic_calming': 'table',
 'type': 'node'}
----
{'_id': ObjectId('58ee9a068a51df7ed62587e6'),
 'created': {'changeset': '37841831',
             'timestamp': '2016-03-15T12:22:06Z',
             'uid': '992708',
             'user': 'Christian Madsen',
             'version': '3'},
 'highway': 'crossing',
 'id': '3643958449',
 'pos': [60.3068157, 5.3518193],
 'surface': 'asphalt',
 'traffic_calming': 'table',
 'type': 'node'}
----
{'_id': ObjectId('58ee9a068a51df7ed62587e7'),
 'created': {'changeset': '37841831',
             'timestamp': '2016-03-15T12:22:06Z',
             'uid': '9927

In [114]:
docs_with_feature('amenity',value='toilets')

41  documents have feature.
Printing first 5 matching documents.
{'_id': ObjectId('58ee99f38a51df7ed61fb603'),
 'amenity': 'toilets',
 'created': {'changeset': '24726672',
             'timestamp': '2014-08-13T15:52:17Z',
             'uid': '1760942',
             'user': 'siggenlh',
             'version': '3'},
 'id': '121742467',
 'pos': [60.3662612, 5.2640783],
 'type': 'node'}
----
{'_id': ObjectId('58ee99f38a51df7ed61fbcc0'),
 'amenity': 'toilets',
 'created': {'changeset': '39999257',
             'timestamp': '2016-06-13T18:36:54Z',
             'uid': '1965308',
             'user': 'FredrikLindseth',
             'version': '4'},
 'id': '171527672',
 'pos': [60.3617322, 5.234721],
 'source': 'local_knowledge',
 'type': 'node'}
----
{'_id': ObjectId('58ee99f38a51df7ed61ffab6'),
 'amenity': 'toilets',
 'created': {'changeset': '468665',
             'timestamp': '2008-08-30T23:18:11Z',
             'uid': '3627',
             'user': 'Vegard Engen',
             'version': '1'

In [111]:
docs_with_feature('toilets',max_docs=50)

48  documents have feature.
{'_id': ObjectId('58ee99f48a51df7ed6204919'),
 'address': {'city': 'Bergen',
             'floor': '1',
             'housenumber': '2a',
             'postcode': '5015',
             'street': 'Nygårdsgaten'},
 'amenity': 'pub',
 'contact': {'facebook': 'https://www.facebook.com/pages/Fincken/153731158012212'},
 'created': {'changeset': '42277189',
             'timestamp': '2016-09-19T18:40:42Z',
             'uid': '673287',
             'user': 'Mariast',
             'version': '7'},
 'gay': 'yes',
 'id': '391946421',
 'name': 'Fincken',
 'opening_hours': 'Mo off; We-Th 19:00-01:30; Fr-Sa 20:00-03:00; Su off',
 'pos': [60.3896606, 5.3230082],
 'toilets': {'wheelchair': 'no'},
 'type': 'node',
 'website': 'http://www.fincken.no/',
 'wheelchair': 'limited'}
----
{'_id': ObjectId('58ee99f48a51df7ed620491a'),
 'address': {'city': 'Bergen',
             'floor': '1',
             'housenumber': '2a',
             'postcode': '5015',
             'street': 'N

In [108]:
for feat in node_features:
    print(feat)
    result = bergen.find_one( { feat: 'asphalt' } )
    if result:
        pprint(result)
        print('------------------------------')

{'_id': ObjectId('58ee9a068a51df7ed62587e5'),
 'created': {'changeset': '32867494',
             'timestamp': '2015-07-25T09:49:05Z',
             'uid': '992708',
             'user': 'Christian Madsen',
             'version': '2'},
 'highway': 'crossing',
 'id': '3643958450',
 'pos': [60.3090102, 5.3528302],
 'surface': 'asphalt',
 'traffic_calming': 'table',
 'type': 'node'}
------------------------------


In [123]:
for key in sorted(node_features,key=node_features.get,reverse=True):
    print(key,node_features[key])

pos 1257508
created.timestamp 628754
id 628754
created.user 628754
created.changeset 628754
type 628754
created.version 628754
created.uid 628754
address.street 83821
address.postcode 83790
address.city 83784
address.housenumber 83778
created_by 7918
source 6537
name 3849
source.date 2634
highway 2528
amenity 2407
natural 2164
leaf_type 1953
shop 1112
material 795
website 739
crossing 598
barrier 573
power 564
wheelchair 532
operator 526
backrest 525
phone 517
public_transport 498
opening_hours 475
tourism 446
entrance 381
level 367
leisure 364
address.floor 317
email 288
colour 269
office 259
place 250
access 242
bus 241
species 226
ele 196
cuisine 194
man_made 193
seamark.type 186
artwork_type 178
note 166
railway 165
start_date 165
recycling_type 162
location 150
capacity 131
description 130
artist_name 128
contact.facebook 124
time 120
no-bergen-playgrounds:id 118
fixme 114
outdoor_seating 109
bicycle 108
traffic_calming 103
foot 101
crossing_ref 100
shelter 97
address.country 92
r

In [125]:
for key in sorted(way_features,key=way_features.get,reverse=True):
    print(key,way_features[key])

node_refs 636080
created.timestamp 52379
id 52379
created.user 52379
created.uid 52379
created.changeset 52379
type 52379
created.version 52379
highway 24684
name 14287
source 11919
building 11420
source.date 10676
nvdb.date 6182
nvdb.id 6077
maxspeed 4982
natural 2941
foot 2199
ref 1997
oneway 1925
surface 1277
amenity 1001
waterway 1000
layer 965
bicycle 886
lanes 817
service 777
landuse 626
created_by 621
address.street 619
barrier 616
address.city 601
address.postcode 599
address.housenumber 598
lit 524
sac_scale 511
bridge 461
railway 456
leisure 448
man_made 447
wetland 417
tunnel 400
access 334
voltage 310
segregated 272
int_ref 258
building.levels 227
electrified 218
note 217
gauge 216
power 193
frequency 187
sport 187
operator 171
wheelchair 171
name.en 165
parking 156
motor_vehicle 156
cycleway 145
height 143
footway 137
website 130
incline 129
fixme 121
motorcar 118
construction 104
psv 101
junction 100
material 97
abutters 91
level 91
wikipedia 90
capacity 86
turn.lanes 84


In [132]:
for a in bergen.find({'type':'way','email':{'$exists': 1}}):
    pprint(a)
    print('-------------')

{'_id': ObjectId('58ee9a118a51df7ed628f832'),
 'address': {'city': 'Bergen',
             'housenumber': '4',
             'postcode': '5014',
             'street': 'Starvhusgaten'},
 'alt_name': 'Telegrafbygningen',
 'building': ['yes',
              {'levels': '4'},
              {'colour': '#674c47'},
              {'material': 'brick'}],
 'created': {'changeset': '39586362',
             'timestamp': '2016-05-26T17:18:50Z',
             'uid': '1965308',
             'user': 'FredrikLindseth',
             'version': '13'},
 'email': 'post@telegrafenbergen.no',
 'id': '24399905',
 'name': 'Telegrafen',
 'node_refs': ['770176638',
               '3946138882',
               '770176635',
               '4209438998',
               '653924',
               '2028740083',
               '3946138883',
               '770176638'],
 'opening_hours': 'Mo-Fr 09:00-20:00; Sa 09:00-18:00',
 'roof': [{'shape': 'flat'}, {'colour': 'black'}],
 'shop': 'mall',
 'start_date': '1927',
 'type': 'way

In [130]:
docs_with_feature('name',value='Telegrafen')

1  documents have feature.
{'_id': ObjectId('58ee9a118a51df7ed628f832'),
 'address': {'city': 'Bergen',
             'housenumber': '4',
             'postcode': '5014',
             'street': 'Starvhusgaten'},
 'alt_name': 'Telegrafbygningen',
 'building': ['yes',
              {'levels': '4'},
              {'colour': '#674c47'},
              {'material': 'brick'}],
 'created': {'changeset': '39586362',
             'timestamp': '2016-05-26T17:18:50Z',
             'uid': '1965308',
             'user': 'FredrikLindseth',
             'version': '13'},
 'email': 'post@telegrafenbergen.no',
 'id': '24399905',
 'name': 'Telegrafen',
 'node_refs': ['770176638',
               '3946138882',
               '770176635',
               '4209438998',
               '653924',
               '2028740083',
               '3946138883',
               '770176638'],
 'opening_hours': 'Mo-Fr 09:00-20:00; Sa 09:00-18:00',
 'roof': [{'shape': 'flat'}, {'colour': 'black'}],
 'shop': 'mall',
 'start_d

In [147]:
docs_with_feature('wifi')

2  documents have feature.
{'_id': ObjectId('58ee99f68a51df7ed620d5c1'),
 'address': {'housename': 'Alrek studentheim (Alrek Student house)',
             'housenumber': '25',
             'postcode': '5009',
             'street': 'Årstadveien'},
 'amenity': 'pub',
 'created': {'changeset': '36398210',
             'timestamp': '2016-01-06T09:23:59Z',
             'uid': '1965308',
             'user': 'FredrikLindseth',
             'version': '3'},
 'id': '1255229011',
 'name': 'Club Alrek',
 'opening_hours': 'Fr',
 'pos': [60.3792644, 5.3588499],
 'type': 'node',
 'wifi': 'free'}
----
{'_id': ObjectId('58ee99f88a51df7ed621a343'),
 'address': {'housename': 'Bergen Kunsthall',
             'housenumber': '5',
             'postcode': '5015',
             'street': 'Rasmus Meyers allé'},
 'amenity': 'cafe',
 'contact': {'facebook': 'https://www.facebook.com/LandmarkBergen'},
 'created': {'changeset': '42284068',
             'timestamp': '2016-09-19T21:26:36Z',
             'uid': '67

In [148]:
docs_with_feature('internet_access',max_docs=50)

29  documents have feature.
{'_id': ObjectId('58ee99f38a51df7ed61fe5df'),
 'amenity': 'cafe',
 'created': {'changeset': '43003014',
             'timestamp': '2016-10-19T08:44:36Z',
             'uid': '1965308',
             'user': 'FredrikLindseth',
             'version': '5'},
 'cuisine': 'coffee_shop',
 'id': '283116334',
 'internet_access': 'no',
 'level': '0',
 'name': 'Kaffehuset Friele',
 'pos': [60.3903313, 5.3328154],
 'type': 'node'}
----
{'_id': ObjectId('58ee99f38a51df7ed62009c2'),
 'amenity': 'cafe',
 'created': {'changeset': '18369044',
             'timestamp': '2013-10-15T13:43:39Z',
             'uid': '1775153',
             'user': 'ovinovre',
             'version': '4'},
 'id': '296552516',
 'internet_access': 'no',
 'name': 'Vikinghytten',
 'note': 'Eies og drives av Turn- og Idrettsforeningen Viking, Bergen. Bygget '
         'av medlemmer i1913.',
 'opening_hours': 'Su 10:30-14:30; Jul off',
 'pos': [60.4213233, 5.3997062],
 'type': 'node'}
----
{'_id': Objec

In [162]:
for key in features:
    if 'amenity' in key:
        print(key,features[key])

source:amenity 3
amenity:no 1
amenity 3423


In [151]:
features['college']

1

In [154]:
docs_with_feature('karaoke')

1  documents have feature.
{'_id': ObjectId('58ee9a098a51df7ed626827a'),
 'address': {'city': 'Bergen',
             'floor': '-1',
             'housenumber': '12',
             'postcode': '5013',
             'street': 'Strandkaien'},
 'amenity': 'pub',
 'contact': {'facebook': 'https://www.facebook.com/Vaagen-Pub-183468781678622/'},
 'created': {'changeset': '39163655',
             'timestamp': '2016-05-07T12:59:33Z',
             'uid': '1965308',
             'user': 'FredrikLindseth',
             'version': '4'},
 'email': 'post@vaagenpub.no',
 'id': '3934152281',
 'karaoke': 'yes',
 'name': 'Vaagen Pub',
 'opening_hours': 'Mo-Tu 11:00-00:00; We 11:00-02:00; Th 11:00-00:00; Fr-Sa '
                  '11:00-03:00, Su 12:00-00:00',
 'outdoor_seating': 'yes',
 'phone': '+47 55 90 03 80',
 'pos': [60.3945637, 5.3229202],
 'type': 'node',
 'website': 'http://vaagenpub.no/'}
----


In [156]:
for feat in features:
    if ':' not in feat and features[feat] < 10:
        print(feat,features[feat])

cycle 1
social_facility 2
local_ref 1
destination 9
taxi 9
cabins 1
length 9
official_name 9
LUFTHAVNTYPE 2
ICAOKODE 2
club 4
kindergarten 1
stars 4
restriction 1
comment 2
outdoor 1
ramp 1
history 4
caravans 3
bys 1
industrial 1
second_hand 7
board_type 6
name_2 1
display 5
leaf_cycle 4
changing_table 5
nat_name 4
abandoned 4
image 2
intermittent 1
odbl 1
advertising 6
sou 1
sex 2
hazard 2
disused 8
tractor 2
ski 5
population 2
tidal 1
musical_instrument 3
cutting 8
delivery 2
craft_beer 8
aquaculture 1
except 1
drive_through 1
shared_with 1
healthcare 4
traffic_sign 1
class 1
goods 5
population_precision 1
mooring 6
place_numbers 3
health_food 1
duration 6
busway 3
repair 1
seats 8
TODO 1
site_type 1
alcohol 1
wifi 2
microbrewery 3
unisex 2
heritage 1
denotation 6
private 3
via 1
moped 8
trailblazed 1
dispensing 3
tactile 1
int_name 8
canoe_rental 1
wires 6
car 4
side 1
track_visibility 1
collection_times 1
bollard 1
noexit 9
salt 1
fireplace 2
gambling 1
communication 1
visibility 5

In [168]:
docs_with_feature('contact.website')

11  documents have feature.
Printing first 5 matching documents.
{'_id': ObjectId('58ee99f38a51df7ed61fdc53'),
 'address': {'city': 'Bergen',
             'housenumber': '4',
             'postcode': '5012',
             'street': 'Nedre Ole Bulls plass'},
 'amenity': 'restaurant',
 'contact': {'website': 'http://www.fridays.no/Display-Nearest-TGIs/tgi-fridays-ole-bulls-plass'},
 'created': {'changeset': '42163487',
             'timestamp': '2016-09-15T03:58:33Z',
             'uid': '1624486',
             'user': 'thetornado76',
             'version': '12'},
 'id': '269953831',
 'level': '1',
 'name': "TGI Friday's",
 'pos': [60.3913702, 5.3238951],
 'type': 'node',
 'wheelchair': 'limited'}
----
{'_id': ObjectId('58ee99f38a51df7ed61fe3fa'),
 'address': {'city': 'Rådal',
             'housenumber': '1',
             'postcode': '5239',
             'street': 'Råtun'},
 'amenity': 'kindergarten',
 'capacity': '99',
 'contact': [{'phone': '+47 55 11 09 40'},
             {'website': 